In [67]:
import numpy as np, rasterio
from tools.tools import nc_to_tif
import xarray as xr
import gzip, dill
import tools.config as config

data_path = f"../../../output/{config.TASK_NAME}/Run_01_GHG_high_BIO_high_CUT_50/output/2025_09_08__09_33_05_RF5_2010-2050/Data_RES5.gz"



In [68]:
with gzip.open(data_path, 'rb') as f:
    data = dill.load(f)

In [60]:
import os
def nc_to_tif(data, da, tif_path: str, nodata_value: float = -9999.0):
    # 仅支持 1D 'cell'
    if "cell" not in da.dims or len(da.dims) != 1:
        raise ValueError(f"维度是 {da.dims}，只支持一维 'cell'。")

    arr1d = da.values.astype(np.float32)
    arr1d = np.where(np.isfinite(arr1d), arr1d, nodata_value)

    # 铺回 2D（保持你的逻辑）
    full_res_raw = (arr1d.size == data.LUMAP_NO_RESFACTOR.size)
    if full_res_raw:
        geo_meta = data.GEO_META_FULLRES.copy()
        arr_2d = np.full(data.NLUM_MASK.shape, nodata_value, dtype=np.float32)
        np.place(arr_2d, data.NLUM_MASK, arr1d)
    else:
        geo_meta = data.GEO_META.copy()
        arr_2d = data.LUMAP_2D_RESFACTORED.copy().astype(np.float32)
        np.place(arr_2d,
                 (arr_2d != data.MASK_LU_CODE) & (arr_2d != data.NODATA),
                 arr1d)

    arr_2d[arr_2d == data.MASK_LU_CODE] = nodata_value
    # 生成有效性掩膜：True=有效，False=无效
    valid_mask = np.isfinite(arr_2d)  # 先按是否为有限数
    # 把 NaN/inf 替换为 nodata 值
    arr_2d = np.where(valid_mask, arr_2d, nodata_value).astype(np.float32)

    meta = geo_meta.copy()
    meta.update(
        count=1,
        dtype="float32",
        nodata=nodata_value,
        compress="deflate",   # 可选：压缩
        predictor=3,          # 浮点预测器
        tiled=True,           # 平铺
        blockxsize=256,       # 块大小（可按需）
        blockysize=256,
    )

    os.makedirs(os.path.dirname(tif_path), exist_ok=True)
    with rasterio.open(tif_path, "w", **meta) as dst:
        dst.write(arr_2d, 1)
        # 写 GDAL 内部掩膜（0=无效, 255=有效）
        dst.write_mask((valid_mask.astype(np.uint8) * 255))

    print(f"✅ 已保存: {tif_path}")

In [61]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high_bio_50\2050\xr_total_cost_carbon_high_bio_50_amortised_2050.nc"

# 输出路径
out_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\xr_total_cost_carbon_high_bio_50_2050.tif"

# 读取和处理
da = xr.open_dataarray(input_path)
da = da.sum(dim=[d for d in da.dims if d != 'cell'])
da = da.where(da >= 1) 
print(np.min(da))
da = da / 1e6
nc_to_tif(data, da, out_tif)

<xarray.DataArray 'data' ()> Size: 4B
array(1.0004827, dtype=float32)
✅ 已保存: N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\xr_total_cost_carbon_high_bio_50_2050.tif


In [65]:
tif_file =  r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\xr_total_cost_carbon_high_bio_50_2050.tif"
with rasterio.open(tif_file) as src:
    bounds = src.bounds
    data = src.read(1, masked=True)
    extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
    raster_crs = src.crs
    nodata = src.nodata
    if nodata is not None:
        data = np.ma.masked_equal(data, nodata)
        
np.nanmin(data), np.min(data)

(1.0004827e-06, 1.0004827e-06)

In [66]:
data

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=-9999.0,
  dtype=float32)

In [33]:
da = xr.open_dataarray(input_path)
da  

<xarray.DataArray 'data' (cell: 168778)> Size: 675kB
[168778 values with dtype=float32]
Coordinates:
  * cell     (cell) int64 1MB 0 1 2 3 4 5 ... 168773 168774 168775 168776 168777

In [63]:
import tools.config as config
from tools.tools import nc_to_tif
import xarray as xr
import dill
import gzip
from joblib import Parallel, delayed

def process_one(env_cat, file_part, base_dir, tif_dir, data):
    """处理一个类别+文件部分，并输出tif"""
    print(f"Processing {env_cat} - {file_part}")

    # 构造输入路径
    if file_part == 'total_cost':
        input_path = f"{base_dir}/{env_cat}/2050/xr_{file_part}_{env_cat}_amortised_2050.nc"
    else:
        input_path = f"{base_dir}/{env_cat}/2050/xr_{file_part}_{env_cat}_2050.nc"

    # 输出路径
    out_tif = f"{tif_dir}/xr_{file_part}_{env_cat}_2050.tif"

    # 读取和处理
    da = xr.open_dataarray(input_path)
    da = da.sum(dim=[d for d in da.dims if d != 'cell'])
    da = da.where(da >= 1, drop=True)
    if 'cost' in  file_part:
        da = da / 1e6
    # 保存到 GeoTIFF
    nc_to_tif(data, da, out_tif)

    return out_tif

base_dir = f"../../../output/{config.TASK_NAME}/carbon_price/0_base_data"
tif_dir = f"../../../output/{config.TASK_NAME}/carbon_price/4_tif"
data_path = f"../../../output/{config.TASK_NAME}/Run_01_GHG_high_BIO_high_CUT_50/output/2025_09_08__09_33_05_RF5_2010-2050/Data_RES5.gz"

# with gzip.open(data_path, 'rb') as f:
#     data = dill.load(f)

# env_categorys =  ['carbon_high', 'carbon_high_bio_50','carbon_low',
#         'carbon_low_bio_10', 'carbon_low_bio_20', 'carbon_low_bio_30', 'carbon_low_bio_40', 'carbon_low_bio_50',
#         'carbon_high_bio_10', 'carbon_high_bio_20', 'carbon_high_bio_30', 'carbon_high_bio_40']

env_categorys =  [ 'carbon_high_bio_50','carbon_high']

file_parts = ['total_cost','cost_ag', 'cost_agricultural_management', 'cost_non_ag', 'cost_transition_ag2ag_diff',
                 'transition_cost_ag2non_ag_amortised_diff','total','price']

tasks = [(env_cat, file_part) for env_cat in env_categorys for file_part in file_parts]

results = Parallel(n_jobs=8)(   # 这里你可以改 n_jobs，比如 8 或 -1 用所有CPU
    delayed(process_one)(env_cat, file_part, base_dir, tif_dir, data)
    for env_cat, file_part in tasks
)

KeyboardInterrupt: 